# Coursework assignment: Data Encryption Standard (DES)
## Applied Cryptography (6G6Z0024_2324_9)
### Killian O'Brien

### Introduction
These questions are based on the idea of question 4.11 from Stallings. The question parts guide you through the application of a single round of the DES encryption algorithm. 

The algorithm is fully specified in appendix C of <a href="https://mmu.on.worldcat.org/oclc/1334132058" target="_blank">Stallings</a>. There you will find flowcharts and explanations of how the DES algorithm works. 

The numbered steps below will take you through the initial preparation of your message $m$ and key $k$, the calculation of the first sub-key $K_1$, and then the calculation of $L_1$ and $R_1$, the outputs of round 1 of DES. 

At the bottom of this notebook you will find Python definitions for lists defining the various transformation functions that form part of DES as described in appendix C. You are encouraged to make use of these, and suitable Python commands to answer the questions. 

### Guidance
In answering these questions, the message $m$ and key $k$ are represented as 64 bit binary strings, i.e. strings of zeros and ones that are 64 characters long. 

The message $m$ you will use is the 64 bit binary string representing the ASCII encoding of your 8-digit university ID number. 

The key $k$ you will use is the 64 bit binary string defined as

    k = '0011000100110010001100110011010000110101001101100011011100111000'
This string is the binary representation of the ASCII encoding of the string `'12345678'` which is calculated as shown in the Python block below.

To help you check your workings and detect any errors the final 64-bit binary string output, which is the concatenation of $L_1$ and $R_1$, for your university ID number, can be found in the table at the following link: <a href="https://killianobrien.github.io/app-cryp2324/schedule/notebooks/des.html" target="_blank">encrypted ID numbers</a>.

In [1]:
k = '12345678'

# bin(ord(x)) gives the python binary string representing the ASCII encoding of 
# the character x. Slicing this from index 2 onwards removes the `0b` binary indication code
# from the front of the string. zfill(8) fills in any leading zeros that were stripped 
# from bin(ord(x)) to give a standard 8 bit binary string.

k = [bin(ord(x))[2:].zfill(8) for x in k]

# the join method joins the eight 8-bit binary strings from the list k with the empty string, i.e. it 
# concatenates the eight 8-bit binary strings into a single 64-bit binary string. 

k = ''.join(x for x in k)
print(k)

0011000100110010001100110011010000110101001101100011011100111000


#### Warning
In the description of DES in Appendix C of Stallings, when referencing the position of a bit in a message block he uses index references beginning at 1. So the left-most bit in a message block is at index 1, the next bit is at index 2 and so on. However when referencing bit positions in binary strings in your Python commands you will need to remember that Python references positions in arrays, strings etc using indexing beginning at 0. So the leftmost bit in a binary string is at index 0, the second bit is at index 1, and so on. You will need to be careful to avoid any *off-by-one* errors. 

### Questions

Prepare a Jupyter notebook and for each question below define the asked for variable. **Make sure you use the exact variable names given in the questions**. For each question you should also prepare a text-cell giving a brief explanation of how you answered the question. The labelling of intermediate variables follows the notation used in figure C.2 of Stallings, pg. 769.

1. Determine the $64$ bit binary string given by the ASCII encoding of your $8$-digit Man Met university ID number. Assign this string to the variable `m` in your notebook. This is your message to be encrypted. 
2. Determine the result of applying the Initial Permutation to your message $m$. Store the resulting 64 bit binary string in the variable `IPm` in your notebook.
3. Determine the left and right halves that are the inputs to the first round. Store these 32 bit binary strings in the variables `L0` and `R0` respectively.
4. Determine the Expansion/permutation of $R_0$ and store the resulting 48 bit binary string in the variable `ER0`.
5. Determine the first subkey $K_1$ and store the 48 bit binary string in the variable `K1`.
6. Determine the XOR operation on the inputs $ER_0$ and $K_1$. Store the resulting 48 bit binary string the variable `A`.
7. Determine the result of applying the S-box substitution/choice operation to your bit string $A$. Store the resulting 32 bit binary string in the variable `SA`.
8. Determine the result of applying the permutation function to $SA$ and store the resulting 32 bit binary string the variable `PSA`.
9. Determine the two 32 bit binary output strings $L_1$ and $R_1$ and store these in the variables `L1` and `R1` respectively. 

### Table data

In [2]:
# Initial Permutation
IP=[
58, 50, 42, 34, 26, 18, 10, 2,
60, 52, 44, 36, 28, 20, 12, 4,
62, 54, 46, 38, 30, 22, 14, 6,
64, 56, 48, 40, 32, 24, 16, 8,
57, 49, 41, 33, 25, 17, 9, 1,
59, 51, 43, 35, 27, 19, 11, 3,
61, 53, 45, 37, 29, 21, 13, 5,
63, 55, 47, 39, 31, 23, 15, 7]


In [3]:
# Expansion permutation
E = [32,1,2,3,4,5,4,5,6,7,8,9,8,9,10,11,12,13,12,13,14,15,16,17, 
16,17,18,19,20,21,20,21,22,23,24,25,24,25,26,27,28,29,28,29,30,31,32,1]

In [4]:
#S-boxes
SBOX = [
# Box-1
[
[14,4,13,1,2,15,11,8,3,10,6,12,5,9,0,7],
[0,15,7,4,14,2,13,1,10,6,12,11,9,5,3,8],
[4,1,14,8,13,6,2,11,15,12,9,7,3,10,5,0],
[15,12,8,2,4,9,1,7,5,11,3,14,10,0,6,13]
],
# Box-2

[
[15,1,8,14,6,11,3,4,9,7,2,13,12,0,5,10],
[3,13,4,7,15,2,8,14,12,0,1,10,6,9,11,5],
[0,14,7,11,10,4,13,1,5,8,12,6,9,3,2,15],
[13,8,10,1,3,15,4,2,11,6,7,12,0,5,14,9]
],

# Box-3

[
[10,0,9,14,6,3,15,5,1,13,12,7,11,4,2,8],
[13,7,0,9,3,4,6,10,2,8,5,14,12,11,15,1],
[13,6,4,9,8,15,3,0,11,1,2,12,5,10,14,7],
[1,10,13,0,6,9,8,7,4,15,14,3,11,5,2,12]

],

# Box-4
[
[7,13,14,3,0,6,9,10,1,2,8,5,11,12,4,15],
[13,8,11,5,6,15,0,3,4,7,2,12,1,10,14,9],
[10,6,9,0,12,11,7,13,15,1,3,14,5,2,8,4],
[3,15,0,6,10,1,13,8,9,4,5,11,12,7,2,14]
],

# Box-5
[
[2,12,4,1,7,10,11,6,8,5,3,15,13,0,14,9],
[14,11,2,12,4,7,13,1,5,0,15,10,3,9,8,6],
[4,2,1,11,10,13,7,8,15,9,12,5,6,3,0,14],
[11,8,12,7,1,14,2,13,6,15,0,9,10,4,5,3]
],
# Box-6

[
[12,1,10,15,9,2,6,8,0,13,3,4,14,7,5,11],
[10,15,4,2,7,12,9,5,6,1,13,14,0,11,3,8],
[9,14,15,5,2,8,12,3,7,0,4,10,1,13,11,6],
[4,3,2,12,9,5,15,10,11,14,1,7,6,0,8,13]

],
# Box-7
[
[4,11,2,14,15,0,8,13,3,12,9,7,5,10,6,1],
[13,0,11,7,4,9,1,10,14,3,5,12,2,15,8,6],
[1,4,11,13,12,3,7,14,10,15,6,8,0,5,9,2],
[6,11,13,8,1,4,10,7,9,5,0,15,14,2,3,12]
],
# Box-8

[
[13,2,8,4,6,15,11,1,10,9,3,14,5,0,12,7],
[1,15,13,8,10,3,7,4,12,5,6,11,0,14,9,2],
[7,11,4,1,9,12,14,2,0,6,10,13,15,3,5,8],
[2,1,14,7,4,10,8,13,15,12,9,0,3,5,6,11]
]

]

In [5]:
# Permutation P
P = [16,7,20,21,29,12,28,17,1,15,23,26,5,18,31,10,
2,8,24,14,32,27,3,9,19,13,30,6,22,11,4,25]

In [6]:
# Permuted choice 1, used in key generation
PC1 = [57,49,41,33,25,17,9,1,58,50,42,34,26,18,10,2,59,51,43,35,27,19,11,3,60,
52,44,36,63,55,47,39,31,23,15,7,62,54,46,38,30,22,14,6,61,53,45,37,29,21,13,5,28,20,12,4]

In [7]:
# Permuted choice 2, used in key generation
PC2 = [14,17,11,24,1,5,3,28,15,6,21,10,23,19,12,4,26,8,16,7,27,20,13,2,41,52,
31,37,47,55,30,40,51,45,33,48,44,49,39,56,34,53,46,42,50,36,29,32]

In [8]:
# Python function for carrying out the xor of two binary strings

def XOR(a,b):
    # a and b should be binary strings of equal length
    c = ''.join(str((int(x) + int(y))%2) for x,y in zip(a,b))
    return c

#example usage
XOR('1100','0101')

'1001'